In [1]:
import os,sys,pymongo
import DSAHelperFunctions as dsa
from pprint import pprint

# Define Local Variables

In [2]:
### Paths and Name for Archive and Location of Slides and Pyramids
archive_name= 'UberSliderSorter'  ### This obviously changes for a given slide set
WSI_root = '/TCGA_MIRROR/'  ##WholeSlide Image Root Directory
### Will eventually make this a list...
## Create Database  Connections
client = pymongo.MongoClient('localhost',27017)
archive_db = client[archive_name]
## Within the ArrchiveDB I will have subcollections for SlideData, MetaData and Combined

In [3]:
## This assumes taht within a given WholeSlideImage directory, the top level directory is the basic grouping for a slide
wsi_dirs = [x for x in os.listdir(WSI_root) if os.path.isdir(os.path.join(WSI_root,x)) ]
print len(wsi_dirs),'folder sets are available'

wsi_dirs.remove('@Recycle')
wsi_dirs.remove('.git')
print wsi_dirs
archive_db['RawSlideData'].create_index('slide_w_path')

10 folder sets are available
['frozen_sections', 'tcga-data.nci.nih.gov', 'MOUSE_GLIOMA', 'EMORY_BIOBANK', 'DJ_SLIDE_DATA', 'WINSHIP_BIOBANK', 'TCGA_FLAT', 'TCGA_FLAT_NONCONFORMING']


u'slide_w_path_1'

In [4]:
def checkSlideStatus( wsiFileList, rootCollxn, dbPtr):
    """This accepts a whole slide image with the full file path, and the root Collection
    and a pointer to the mongoDatabase housing the raw slide data, it will then update/amend as needed"""
    slides_processed = dup_slide = newly_processed = rescanned_slides =  bad_slides = 0
    for wsi in wsiFileList:
        ### Going to get file Stats
        qry = dbPtr['RawSlideData'][rootCollxn].find_one( {'slide_w_path':wsi})
        if not qry:  ## Means I couldn't find this file loaded
            ###Double check it's not an ivnalid filee
            
            (openslide_could_open, sldMetadata) = dsa.OpenslideGetImageMetadata( wsi)
            
            if openslide_could_open:
                ### Need to inject in some other keys
                prep_type = "Unknown"
                sldMetadata['slide_w_path'] = wsi
                sldMetadata['file_size'] = os.path.getsize(wsi)
                sldMetadata['slide_name'] = os.path.basename(wsi)
                sldMetadata['scanProperties'] = dsa.clean_openslide_keys( sldMetadata['scanProperties'])
                newly_processed +=1 
                dbPtr['RawSlideData'][rootCollxn].insert_one(sldMetadata)
            else:
                sldMetadata['slide_w_path'] = wsi
                sldMetadata['OpenslideFailedOpen'] = True

                dbPtr['LoadErrorDB'][rootCollxn].insert_one(sldMetadata)
                dbPtr['RawSlideData'][rootCollxn].insert_one(sldMetadata)
                
                bad_slides +=1

        else:
            ### See if the other slide matches  #print "Image seems to have been loaded?"  This is if I find two slides with same name with difts sizes
            rescanned_slides +=1
        slides_processed +=1        
        output = "Total Processed: %d  Newly Processed: %d Dup Slides or Already Loaded: %d  RESCANNED Slides %d badSlides %d" % (slides_processed, newly_processed, dup_slide, rescanned_slides, bad_slides )
        dsa.LinePrinter(output)
    print ### Need this at the end of the block to advacne the dsa/LinePrinter comment abobe

In [5]:
reload(dsa)

### For debugging I'm going to load the entire slide list into memory...
for wsiRD in wsi_dirs:
    ### I am going to use the Top Level Directory i.e. wsiRD as the primary collection/folder
    ### and grab all the children files
    print wsiRD
    wsiFileList = dsa.get_filepaths(os.path.join(WSI_root,wsiRD))
    checkSlideStatus(wsiFileList, wsiRD, archive_db)
    ### Now iterate through the fileList and update/amend files as needed if they are not in the database
    
### u'openslide.objective-power': u'40',
#paths = [os.path.join(path,fn) for fn in next(os.walk(path))[2]]

frozen_sections

tcga-data.nci.nih.gov
Total Processed: 107455  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 107455 badSlides 0
MOUSE_GLIOMA
Total Processed: 284  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 284 badSlides 0
EMORY_BIOBANK
Total Processed: 232  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 232 badSlides 0
DJ_SLIDE_DATA

WINSHIP_BIOBANK
Total Processed: 10  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 10 badSlides 0
TCGA_FLAT
Total Processed: 32307  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 32307 badSlides 0
TCGA_FLAT_NONCONFORMING
Total Processed: 288  Newly Processed: 0 Dup Slides or Already Loaded: 0  RESCANNED Slides 288 badSlides 0


In [ ]:
wsiFileList = dsa.get_filepaths(os.path.join(WSI_root,'TCGA_FLAT'))
print len(wsiFileList)

In [ ]:
checkSlideStatus(wsiFileList,'TCGA_FLAT', archive_db)

In [ ]:
reload(dsa)
wsi='/TCGA_MIRROR/TCGA_FLAT/coad/TCGA-A6-5656-01Z-00-DX1.8a8ebf52-8217-4288-8886-7eefa6cdfdca.svs'
(openslide_could_open, sldMetadata) = dsa.OpenslideGetImageMetadata( wsi)
print openslide_could_open
print sldMetadata

In [ ]:
#             if qry['file_size'] != fs:
#                 #print "File size mismatch??",fs,qry['file_size'],qry['slide_w_path'],sld
#                 load_errors_db['rescanned_slides'].insert_one( {'loaded_slide': qry['slide_w_path'], 'rescanned_slide': sld}             )
#                 rescanned_slides +=1 
#             else:
#                 dup_slide +=1

In [ ]:
m = archive_db['RawSlideData']

In [ ]:
m.

In [ ]:
### This is a composite view that combines the pyramid and original NDPI tables
### This may eventually link in other information abut the slides as well
combined_db = client[archive_name]
### The combo view combines data from the slide and pyramid database...
pyr_collections = [x for x in combined_db.collection_names() if 'PyramidData' in x]

## Iterate through the pyramid table for each collection
iip_base_url = 'http://node15.cci.emory.edu/cgi-bin/iipsrv.fcgi'
    
for pyr_c in pyr_collections:
    ## Now iterate through the list of pyramids within each group
    print pyr_c
    
    cur_pyrgrp_ptr = combined_db[pyr_c].find()
    for pyr in cur_pyrgrp_ptr:
        #print pyr
        #sys.exit()
        ### Now I am only going to grab a subset of the fields... not all are really needed by the CDSA currently
        ## The SlideURL and thumbnail are built frmo the other fields...
        #'http://node15.cci.emory.edu/cgi-bin/iipsrv.fcgi?FIF=/bigdata2/PYRAMIDS/CDSA/DLBC_Frozen/nationwidechildrens.org_DLBC.tissue_images.Level_1.248.0.0/TCGA-FA-8693-01A-01-BS1.263ab76a-8168-4f72-8147-8c2a44d0948b.svs.dzi.tif&WID=200&CVT=jpeg
        thumbnail_url = "%s?FIF=%s&WID=200&CVT=jpeg" % (iip_base_url, pyr['pyramid_w_path'] )
        
        slide_url = "%s?DeepZoom=%s.dzi" % (iip_base_url, pyr['pyramid_w_path'] )
        #print slide_url
        slide_basename = pyr['pyramid_name'].split('.')[0] ## This basically just returns the part of the slide before the dot.. removes the extensions and UID
        folder = pyr_c.replace('PyramidData.','')
        
        prep_type = 'Unk'
        
        cdsa_rec = { 'slide_id': pyr['_id'], 'slide_url': '', 'annotation': False,  'thumbnail': 'TBD', 
                      'image_width': pyr['width'], 'image_height': pyr['height'], 'pyramid_filename': pyr['pyramid_name'],
                       'tumor_type' : folder, 'prep_type': prep_type, 'slide_url': slide_url,
                       'thumbnail_image': thumbnail_url, 'slide_name': slide_basename
                   }
        #print cdsa_rec
        print folder
        combined_db['Merged'][folder].insert_one(cdsa_rec)
        
        
        #sys.exit()

In [ ]:
# ### CHECK OUT FILE 2620 +-10 this is for PRAD
# pyramid_root = '/PYRAMIDS/EmoryBioBank/'



# for sc in ndpi_collections:
#     #print "ANALYZING", sc,"Tumor Type"
#     ### First get the slide collection data, this is at the tumor type level
#     ## The pyramid collections are organized a bit differently, so I need to map the collection names between the systems
#     print sc
#     ndpi_files = slide_db[sc].find()
#     #print ndpi_files
#     ndpi_w_pyramid = 0
#     needs_pyramid = 0
#     for ndpi in ndpi_files:
#         #print ndpi
#         slide_name = ndpi['slide_name']
#         prep_type = ndpi['prep_type']
#         #print svs
#         pyramid_name = slide_name+'.dzi.tif'
#         print pyramid_name
        
# ###        #print "Looking for",pyramid_name,pyramid_collection_name
#         pyr_query =  slide_db['PyramidData'][sc].find_one({'pyramid_name': pyramid_name})
#         if pyr_query:
#             #print pyr_query, slide_name, prep_type
#             ndpi_w_pyramid +=1
#         else:
#             needs_pyramid +=1
#             slide_w_path = ndpi['slide_w_path']
#             prep_type = 'UNK'
#             ### doing a set of nonintuitive transformatinos for the pyramid output directory
            
#             archive_dir = os.path.dirname(slide_w_path).replace(ndpi_root,'')
#             #archive_dir = archive_dir
#             ## #Need to make sure output directory is created or the conversion failes
#             output_pyr_dir = os.path.join(pyramid_root, archive_dir )
#             print output_pyr_dir
#             if not os.path.isdir(output_pyr_dir):
#                 print "Need to make output dir!",output_pyr_dir
                
#                 os.makedirs(output_pyr_dir)
#                 vips_fp.write( "mkdir -p "+output_pyr_dir+"\r\n" )
#                 ### This code may need some thought... I am running this process as dagutman or root... may
#                 ### Screw up the file system
            
#             pyr_w_path  = os.path.join( output_pyr_dir, pyramid_name)
                        
                        
#             vips_fp.write( "vips im_extract_bands "+slide_w_path+pyr_w_path+":jpeg:75,tile:256x256,pyramid,,,,8 0 3\r\n" )

#     print sc,"needs",needs_pyramid,"and these svs files seem to have pyramids",ndpi_w_pyramid


In [ ]:
def check_for_pyramid(slide_w_path,base_folder):
    """This will take an input string, and given a slide, will search for a corresponding pyramid tiff, if not it will output
    the syntax to create one"""
    print slide_w_path,"is the slide I am on"
    
    slide_name = os.path.basename(slide_w_path)
    slide_dir = os.path.dirname(slide_w_path)
    pyramid_name = slide_name+'.dzi.tif'
    print pyramid_name,slide_dir,slide_name
    
    ### First check and see if I have already loaded a pyramid for this image into the corresponding table
    pyr_query =  archive_db['PyramidData'][base_folder].find_one({'pyramid_name': pyramid_name})
    print pyr_query
    if pyr_query:
        print "Pyramid EXISTS!"
        return True
    else:
        ### Need to either load or create the pyramid
    
        ##In this case I am assuming there is a one to one correspondence between where I want pyramids to
        ##Live and where I want the original slides to live; with the only diff being the root of the path being
        # NDPI_ROOT or PYRAMID_ROOT    The files also have a 1:1 correspondence except for the extension being .dzi.tif
        pyramid_dir = os.path.dirname(slide_w_path).replace(ndpi_root,pyramid_root)
        pyramid_name = slide_name+'.dzi.tif'
        print pyramid_dir, slide_dir
        pyr_w_path  = os.path.join( pyramid_dir, pyramid_name)
        
        if not os.path.isfile(pyr_w_path):
            print "vips im_extract_bands "+slide_w_path+' '+pyr_w_path+":jpeg:75,tile:256x256,pyramid,,,,8 0 3\r\n"
            vips_fp.write( "vips im_extract_bands "+slide_w_path+' '+pyr_w_path+":jpeg:75,tile:256x256,pyramid,,,,8 0 3\r\n" )
        else:
            print "I NEED TO LOAD THIS INTO THE DATABASE"
            fs = os.path.getsize(pyr_w_path)
            (openslide_could_open, width, height, filesize, orig_resolution, slide_name,md5, pyramid_properties) = cdsa_helpers.openslide_test_file_mongo( pyr_w_path, 'pyramid', client)

            pyramid_metadata = { 'pyramid_w_path': pyr_w_path, 'pyramid_name': pyramid_name, 'file_size':fs, 'width':width, 'height':height,
                                 'pyramid_properties': cdsa_helpers.clean_openslide_keys ( pyramid_properties), 'slide_md5': md5, 
                                 'conversion_info': 'TBD'
                                 }
                #print slide_metadata
            archive_db['PyramidData'][base_folder].insert_one(pyramid_metadata)


In [ ]:
check_for_pyramid('/TCGA_MIRROR/EMORY_BIOBANK/Emory_BioBank/GBM_GLIOMA_PROJECT/11-1_PNC2.ndpi','GBM_GLIOMA_PROJECT')

In [ ]:
#Displays the current collections in the database
print archive_db.collection_names(False)
#archive_db['SlideData.DigitalBioBank'].drop()  ## If I need to drop anything

In [ ]:
print curr_svs_slide_list[4]

In [ ]:
print archive_db.collection_names()

pyramid_folders = [x for x in archive_db.collection_names() if 'PyramidData' in x]
print pyramid_folders